In [ ]:
using IntervalArithmetic, LinearAlgebra, Serialization

In [ ]:
setprecision(1024)

LinearAlgebra.norm(v::Vector) = sqrt(sum(v.^2))

function enlarge(x::Interval{BigFloat})
    isguaranteed(x) || error("interval is not guaranteed")
    return interval(BigFloat(inf(x), RoundDown), BigFloat(sup(x), RoundUp))
end

function Base.inv(P::Bidiagonal{Interval{Float64}, Vector{Interval{Float64}}})
    if P.uplo == 'U'
        C = -P.ev./P.dv[1:end-1]
        invC = UpperTriangular(zeros(eltype(P), size(P)))
        for i in 1:size(P)[1]
            invC[i,i:end] = cumprod([one(eltype(P)); C[i:end]])
        end
        return UpperTriangular(invC ./ P.dv')
    else
        return LowerTriangular(inv(P')')
    end
end    

function Base.:*(D::Diagonal, v::Vector)
    return diag(D).*v
end

function Base.:*(D::Diagonal, A::Matrix)
    return diag(D).*A
end

function Base.:*(A::Matrix, D::Diagonal)
    return A.*diag(D)'
end

# rigorous upper bound of the 2-norm of a matrix
function op_norm(A)
    if size(A) == (2,2)
        Z = sqrt(sum(A.^2) + sqrt(((A[1,2]+A[2,1])^2+(A[1,1]-A[2,2])^2)*((A[1,2]-A[2,1])^2+(A[1,1]+A[2,2])^2)))/sqrt(interval(2))
        if isguaranteed(Z)
            return interval(sup(Z))
        else
            return Z
        end
    else
        B = A'A
        Λ̄, V̄ = eigen(mid.(B))
        Λ = inv(interval.(V̄))*B*interval.(V̄)
        all(isguaranteed.(Λ)) || error("matrix not guaranteed")
        return sqrt(sqrt(interval(maximum(sup.(sum(abs.(Λ), dims = 1))))*interval(maximum(sup.(sum(abs.(Λ), dims = 2))))))
    end
end

In [ ]:
n = 3000
setprecision(8192)
b₁ = deserialize("b1_k4")
setprecision(precision(mid.(b₁)))
b₀ = interval(BigFloat, 0.0)
b = zeros(Interval{BigFloat},n+6)
b[1:2] = [b₀, b₁]
for k = 2:n+4
    b[k+1] = interval(4)+(interval(BigFloat, k-1)/b[k])-b[k]-b[k-1]
end
setprecision(1024)
b = b[2:end]
a = enlarge.(sqrt.(b))
b = enlarge.(b);

In [ ]:
c⁺ = deserialize("c_plus")
c⁻ = deserialize("c_minus")
C_α = deserialize("C_alpha")
θ = deserialize("theta")
Cd = interval(1)/C_α/sqrt(interval(1)-θ^2);

In [ ]:
α = interval.(Float64, interval.(collect(2:2:n+2))./a[2:2:n+2])
β = interval.(Float64, a[2:2:n+2].*a[3:2:n+3].*a[4:2:n+4])
P̄ = Bidiagonal([interval(1); α[1:end-1]],  [interval(0); β[1:end-2]], :U)
P̄⁻¹ = Matrix(interval.(Float64, inv(P̄)))
α = interval.(Float64, interval.(collect(1:2:n+2))./a[1:2:n+1])
β = interval.(Float64, a[1:2:n+2].*a[2:2:n+2].*a[3:2:n+3])
A = α.*P̄⁻¹
B = β.*P̄⁻¹;

In [ ]:
c_α = op_norm([op_norm(A) β[end-1]*norm(A[:,end])*Cd/interval(n)^interval(3//4) ;
        interval(0) sqrt(c⁺/c⁻)/(interval(1)-θ)*interval((n+3)//(n+2))^interval(3//4)])

In [ ]:
c_β = op_norm([op_norm(B) β[end-1]*norm(B[:,end])*Cd/interval(n)^interval(3//4) ;
        interval(0) c⁺^2/(interval(1)-θ)*interval((n+3)//(n+2))^interval(3//4)/interval(3)])

In [ ]:
c₀ = c_α + c_β

In [ ]:
α = interval.(Float64, interval.(collect(1:2:n+3))./a[1:2:n+3])
β = interval.(Float64, a[1:2:n+3].*a[2:2:n+4].*a[3:2:n+5])
P̄ = Bidiagonal(α[1:end-1],  β[1:end-2], :U)
P̄⁻¹ = Matrix(interval.(Float64, inv(P̄)))
α = interval.(Float64, interval.(collect(2:2:n+2))./a[2:2:n+2])
β = interval.(Float64, a[2:2:n+2].*a[3:2:n+3].*a[4:2:n+4])
A = α.*P̄⁻¹
B = β.*P̄⁻¹;

In [ ]:
c_α = op_norm([op_norm(A) β[end-1]*norm(A[:,end])*Cd/interval(n+1)^interval(3//4) ;
        interval(0) sqrt(c⁺/c⁻)/(interval(1)-θ)*interval((n+4)//(n+3))^interval(3//4)])

In [ ]:
c_β = op_norm([op_norm(B) β[end-1]*norm(B[:,end])*Cd/interval(n+1)^interval(3//4) ;
        interval(0) c⁺^2/(interval(1)-θ)*interval((n+4)//(n+3))^interval(3//4)/interval(3)])

In [ ]:
c₁ = c_α + c_β

In [ ]:
c = max(c₀, c₁)

In [ ]:
serialize("c0", c₀)
serialize("c1", c₁)
serialize("small_c", c)